In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
##  Modèles de régression avec scikit-learn : Régression linéaire simple
# ------------------------------------------------------------------------------------------------------------------------------

#Rappel : 
# Etant donnée deux variables X et Y, dites respectivement variable explicative et variable expliquée,
# la régression linéaire simple consiste en la modélisation d'un lien entre la variable explicative et la variable expliquée
# permettant de prédire la valeur de Y lorsque la valeur de X est mesurée

# Le modèle usuel de régression linéaire simple est le suivant:  y = β0 + β1 x + ε 
        # avec : 
        # y  est une variable, quantitative continue, à expliquer (ou variable dépendante),
        # x  est une variable, quantitative continue, explicative,
        # ε  est un terme d’erreur aléatoire de loi normale d’espérance nulle et d’écart-type σ.


# - A ------------------------------------------------------------------------------ Preparation des df

# - 1 --------------------------------------- import

import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import  train_test_split , cross_validate
from sklearn.linear_model import LinearRegression 

df = pd.read_csv('automobiles.csv')  

df.head(1)

df.describe() #les statistiques descriptives élémentaires des colonnes de df.

df.plot.scatter ( 'curb-weight' , 'price' )   #tracer une courbe 


# - 2 --------------------------------------- Créer les vecteurs prix et mv
prix  = df['price']
mv  = df[ 'curb-weight' ]

# - 2 bis --------------------------------------- Créer les vecteurs prix et mv : shape
mv.shape  #(159,)
mv = np.expand_dims(mv, axis=1) #new dim -> new shape (159, 1)
mv =  np.hstack( (mv, np.ones(mv.shape)))   #array([[mv(1), 1.000e+00],[mv(2), 1.000e+00]]
mv.shape  #(159,2)

# - B ------------------------------------------------------------------------------ Modelisation
# - 3 --------------------------------------- Instanciez un régresseur slr -> classe LinearRegression
slr = LinearRegression ()
slr.fit(mv,prix) #entrainement


# - 4 --------------------------------------- intercept et l'unique coefficient
#intercept_ : à l'ordonnée à l'origine  
#coef_ : la pente de la droite dans le cas d'une régression linéaire simple
slr.intercept_ , slr.coef_ 
#prix = intercept_ + slr.coef_  x curb-weight.
#prix = -15378.2 + 10.9 x curb-weight.
#y    = b        + a    x curb-weight.


# - C ------------------------------------------------------------------------------coefficient de détermination (R²) 

# - 5 --------------------------------------- Evaluez le modèle slr par validation croisée à 4 échantillons
cross_validate(slr, mv, prix, return_train_score=True, cv=4)

# moyenne des scores obtenus sur les échantillons de test
cross_validate(slr, mv, prix, return_train_score=True, cv=4)['test_score'].mean() #
#


# - 7 --------------------------------------- prediction 
#Calculez les valeurs ajustées dans un tableau appelé pred_prix, puis calculez les résidus dans residus
pred_prix = slr.predict(mv)
residus = pred_prix - prix
residus.describe()


# - 8 ---------------------------------------  Afficher 1 droite de régression
#Afficher un nuage de point entre les valeurs de 'curb-weigtht' et les prix.
# & la droite de régression grâce aux prédictions pred_prix.
plt.figure(figsize= (10,8))
plt.scatter(mv['curb-weight'], prix, color = 'darkblue')     #x / y reel 
plt.plot(mv, pred_prix ,color='k')                           #x / y predict


# - 9 ---------------------------------------  Importer la fonction f_regression

from sklearn.feature_selection import f_regression

print('F-score:', f_regression(mv, prix)[0], 'p-value :', f_regression(mv, prix)[1])

## p-value ~ 0 donc la variable mv très significative.


# - 10 --------------------------------------  l'erreur quadratique (RMSE en anglais)
#Le RMSE est la racine carrée de la moyenne des résidus au carré.

def rmse(predictions, targets):  #fct RMSE
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse(pred_prix, prix)

#plus l'erreur quadratique moyenne est proche de 0, plus précises sont les prédictions.

from sklearn.model_selection import cross_val_predict
pred_prix2 = cross_val_predict(slr, mv, prix, cv=4) 
rmse(pred_prix2, prix)

----

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
##  Modèles de régression avec scikit-learn : Régression linéaire multiple
# ------------------------------------------------------------------------------------------------------------------------------


#rappelle que le modèle usuel de régression linéaire multiple est le suivant:
                        #   yi=β0+β1xi,1+β2xi,2+⋯+βpxi,p+εi
                        #     =β0+∑j=1pβjxi,j+εi
#où :
#yi est une variable, quantitative continue, à expliquer (ou variable dépendante),
#x1,…,xp sont des variables, quantitatives continues, explicatives,
#les εi sont des termes d'erreur aléatoire de loi normale d'espérance nulle et d'écart-type σ.


# - A ------------------------------------------------------------------------------ Preparation 

# - 1 --------------------------------------- import
import numpy as np
import pandas as pd

from sklearn import model_selection, preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline 

import scipy.stats as stats
import seaborn as sns

df = pd.read_csv("automobiles.csv")




# - B ------------------------------------------------------------------------------ Modelisation (la totalité des features)


# - 1 --------------------------------------- target, data
target = df['price']
data = df.drop('price',axis =1)


# - 2 --------------------------------------- ensemble d'apprentissage et un ensemble de test
X_train,X_test, y_train,  y_test = train_test_split (data,target,test_size = 0.2 )


# - 3 --------------------------------------- modelisation + fit 
lr = LinearRegression()    #Instanciez un régresseur lr de la classe LinearRegression
lr.fit(X_train , y_train)  #fit


# - 4 --------------------------------------- Intercept et coefficients 
coeffs = list(lr.coef_)
coeffs.insert(0, lr.intercept_)

feats = list(data.columns)
feats.insert(0, 'intercept')

pd.DataFrame({'valeur estimée': coeffs}, index = feats)




# - C ------------------------------------------------------------------------------ Score 

# - 5 --------------------------------------- score échantillon d'apprentissage
lr.score(X_train, y_train)      # score du modèle sur l'échantillon d'apprentissage

cross_val_score(lr,X_train,y_train).mean()   #score obtenu par validation croisée

#si les scores sont trop differents pb (surentrainement ) voir score echantillion tests pour determiner le pb


# - 7 --------------------------------------- score du modèle sur l'ensemble de test
lr.score(X_test,y_test)  




# - D ------------------------------------------------------------------------------ Graph 

# - 8 --------------------------------------- graphique pred_test  / 

pred_test = lr.predict(X_test)  
plt.scatter(pred_test, y_test)  # y pred/y reeel
plt.plot((y_test.min(),y_test.max()), (y_test.min(),y_test.max()))  #y reel / y reel avec un delta maximiser

#analyse si point disperses, residus


# - 9 --------------------------------------- Prediction et residus
pred_train = lr.predict(X_train)
residus = pred_train - y_train


# - 10 -------------------------------------- nuage des points , les résidus en fonction des valeurs de y_train
plt.scatter(y_train, residus, color = '#980a10', s=15)
plt.plot((y_train.min(),y_train.max()), (0,0), lw=3, color = '#0a5798')




# - D ------------------------------------------------------------------------------ ajustement d'une distribution 


# - 11 --------------------------------------- Centrez normaliser 
#Centrez réduire les résidus dans un vecteur appelé residus_norm
residus_norm = (residus-residus.mean())/residus.std()


# - 12 -------------------------------------- QQ-Plot  (courbe : un diagramme Quantile-Quantile)
stats.probplot(residus_norm, plot=plt)
plt.show()


# - 13 -------------------------------------- correlation des features ( data) 
# on parle de corrélation si > 80
plt.figure(figsize=(13,13))
sns.heatmap(df.corr(),  annot=True, cmap="RdBu_r", center =0);

#ou 
data.corr()

# - 14 -------------------------------------- Affichez une matrice des nuages de points (les var les + correle a la target)
#entre les variables 'curb-weight', 'horsepower', 'highway-mpg', 'height', 'bore', 'width' et 'price' de df.

sns.pairplot(df[['curb-weight', 'horsepower', 'highway-mpg', 'height', 'bore', 'width', 'price']])



#analyse des courbes entre les datas et la target




# - E ------------------------------------------------------------------------------ ajustement d'une distribution 


# - 15 -------------------------------------- Créez une liste "signif_features" avec var correlé 
#qui contient les noms des variables 'curb-weight', 'horsepower', 'bore', 'width'  var correlé
signif_features = ['curb-weight', 'horsepower', 'bore', 'width']  #list de str colomnnes choisie


# - 16 -------------------------------------- modèle de régression lr2 + l'entraîner + score sur ensemble de train
lr2 = LinearRegression()
lr2.fit( X_train[signif_features], y_train)   #fit avec xtrain des colonnes choisie
lr2.score(X_train[signif_features], y_train)  #score avec xtrain des colonnes choisie

lr2.score(X_test[signif_features], y_test)  #score sur ensemble de test

#La sélection des variables a permis de réduire l'over-fitting (sur-apprentissage)
# - > Le modèle est plus consistant et plus robuste




# - F ------------------------------------------------------------------------------ fonction SelectKBest
#Sélectionner, à partir d'un jeu de données et d'une variable cible, 
#les k variables les plus significatives par rapport à une fonction de score utilisée, (exemple :la fonction f_regresison)


# - 1 --------------------------------------- SelectKBest

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

sk = SelectKBest(f_regression, k=3) #selecteur de var (3var )

sk.fit(X = data, y = target)   #fit

data.columns[sk.get_support()]  #Affiche noms de colonnes sélectionnées par le sélecteur


# - 2 -------------------------------------- méthode transform

sk_train = sk.transform( X_train ) #transform :transforme en table numpy    
sk_test = sk.transform ( X_test)   # les colonnes selectionnée par le selecteur SelectKBest


# - 3 -------------------------------------- modèle de régression linéaire

lr10 = LinearRegression ()             #modele
lr10.fit   ( sk_train , y_train )             #fit
print (lr10.score ( sk_train , y_train ))   #score
print (lr10.score ( sk_test , y_test  ))   #score


# - G ------------------------------------------------------------------------------ fonction SelectFromModel 
#sélectionner les variables importantes d'un dataframe, 
#à partir d'un modèle crée possédant un attribut coef_ ou feature_importances


# - 1 --------------------------------------- classe SelectFromModel

from sklearn.feature_selection import SelectFromModel   #import de la foncction 

lr = LinearRegression()    # modèle de régression linéaire lr

sfm = SelectFromModel(lr)  # sélecteur sfm SelectFromModel, à partir du modèle lr créé

scaler = preprocessing.StandardScaler().fit(X_train)  #appel de fct de normalisation

X_train_scaled = scaler.transform(X_train)  # -> table numpy + norm

X_test_scaled = scaler.transform(X_test)    # -> table numpy+ norm

sfm_train = sfm.fit_transform(X_train_scaled, y_train)   # -> table numpy + norm + colonne choisie par selecteur

sfm_test = sfm.transform(X_test_scaled)                 # -> table numpy+ norm

# - 2 --------------------------------------- nom des colonnes sélectionnés par sfm
data.columns[sfm.get_support()]   #retopurne les noms des col choisies 


# - 3 --------------------------------------- modèle + fit + score
sfmlr = LinearRegression()        #   modele de 
sfmlr.fit(sfm_train, y_train)     # fit

print(sfmlr.score(sfm_train, y_train))   #score 
print(sfmlr.score(sfm_test, y_test))     #score 

----

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
##  Modèles de régression avec scikit-learn : Régression linéaire régularisée : Ridge, Lasso 
# ------------------------------------------------------------------------------------------------------------------------------


#rappelle que le modèle est le suivant:
        #  régression linéaire est modélisée par l'équation:      yi=β0+β1xi,1+β2xi,2+⋯+βpxi,p+εi

        #    forme matricielle :                                  Y=Xβ+ϵ

#La Régression régularisée : 
# L'objectif est d'éviter le sur-apprentissage 

# - A ------------------------------------------------------------------------------ Régression Ridge
#La régression Ridge consiste en l'ajout d'une contrainte sur les coefficients 
#lors de la modélisation pour maîtriser l'amplitude de leurs valeurs.
#    minβ(||||Y−Xβ||||2+α||||β||||2)    avec α ou le coefficient de pénalité


# - 1 --------------------------------------- import

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline

#impoort csv
df = pd.read_csv('automobiles.csv')

#connaite son df
df.head(2)
df.describe()
df.shape # (159, 16)



# - 2 --------------------------------------- traitement du df

#normalisation
scaler = preprocessing.StandardScaler().fit_transform(df)  #fct de normalisation 
df[df.columns] = pd.DataFrame(scaler)  #normalisation

#cut the df  
target = df['price']                #y = target
data = df.drop('price', axis = 1)   #x = data 

X_train, X_test,  y_train, y_test = train_test_split (data, target , test_size = 0.2) # ensemble d'apprentissage, de test


# - 3 ---------------------------------------  classe Ridge 

from sklearn.linear_model import RidgeCV  #import modele RidgeCV 

ridge_reg = RidgeCV(alphas= (0.001, 0.01, 0.1, 0.3, 0.7, 1, 10, 50, 100))  #le modele choisit le param alpha le + mieux

ridge_reg.fit( X_train,   y_train)  #fit du modele 

# - 4 --------------------------------------- param alpha_
ridge_reg.alpha_   #Affiche le  α  retenu par le modèle


# - 5 --------------------------------------- SCore des ensembles
print("score train :", ridge_reg.score(X_train, y_train))       #fit de l ensemble train 
print("score test :", ridge_reg.score(X_test, y_test))       #fit de l ensemble trtestain 

#analyse des scores et deduction si le modèle overfit

# - 6 --------------------------------------- Predict + mean_squarred_error

ridge_pred_train= ridge_reg.predict ( X_train )    #predict  
ridge_pred_test = ridge_reg.predict ( X_test )    #predict

print (mean_squared_error(ridge_pred_train, y_train))   #mean_squarred_error
print (mean_squared_error(ridge_pred_test, y_test))   #mean_squarred_error




# - B ------------------------------------------------------------------------------ Régression Lasso
#La régression Lasso est similaire à la régression Ridge, 
#la seule différence entre les deux étant la contrainte sur la norme L1 avec Lasso, plutôt que sur la norme L2.


# - 1 ---------------------------------------  classe lasso 

from sklearn.linear_model import Lasso   #imoport modele

lasso_r = Lasso(alpha=1)     #modele lasso instancier

lasso_r.fit(X_train, y_train)  #fit du modele  sur l ensemble de train


# - 2 ---------------------------------------   coef_ 
lasso_r.coef_  #attribut coef

#analyse du resultat si modele =0 alpha doit etre changé


# - 3 ---------------------------------------  classe lasso new modele

lasso_reg = Lasso(alpha=1)     #modele lasso instancier

lasso_reg.fit(X_train, y_train)  #fit du modele  sur l ensemble de train

# - 4 ---------------------------------------  courbe du coef 

plt.plot(range(len(data.columns)), lasso_coef)
plt.xticks(range(len(data.columns)), data.columns.values, rotation=70)
plt.show()

#analyse des variables choisie ( >0)


# - 5 ---------------------------------------  score R2 sur l'ensemble d'apprentissage et l'ensemble de test
print("score train:",lasso_reg.score(X_train,y_train))
print("score test :", lasso_reg.score(X_test,y_test))


# - 6 --------------------------------------- Predict + mean_squarred_error
lasso_pred_train = lasso_reg.predict(X_train)
lasso_pred_test = lasso_reg.predict(X_test)

print("mse train:", mean_squared_error(lasso_pred_train, y_train))
print("mse test:", mean_squared_error(lasso_pred_test, y_test))


 
# - C ------------------------------------------------------------------------------ lasso_path 
# produit les coefficients éstimés corrrespondant aux différents  αα  qu'elle reçoit en arguments

# - 1 ---------------------------------------   lasso_path

from sklearn.linear_model import lasso_path    #imoport fonction

mes_alphas = (0.001,0.01,0.02,0.025,0.05,0.1,0.25,0.5,0.8,1.0)   #liste d alplha

alpha_path, coefs_lasso, _ = lasso_path(X_train, y_train, alphas=mes_alphas)  #sur l ensemble de train on applique le param

coefs_lasso.shape

# - 2 ---------------------------------------  graphique
import matplotlib.cm as cm

plt.figure(figsize=(10,7))

for i in range(coefs_lasso.shape[0]):
    plt.plot(alpha_path, coefs_lasso[i,:], '--')

plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Lasso path')
plt.show()

#analyse 
#Le graphique nous montre qu'à partir de  α=0.3, plus que 3 variables seulement sont sélectionnées par le modèle, 
#et qu'à  α=1 , effectivement, tous les coefficients sont nuls.


# - 3 ---------------------------------------  modèle LassoCV

from sklearn.linear_model import LassoCV   #import classe

model_lasso = LassoCV(cv=10).fit(X_train, y_train)  #modele + fit

alphas = model_lasso.alphas_  #param alpha


#Figure Mean square error pour chaque échantillon
plt.figure(figsize = (10,8))  

plt.plot(alphas, model_lasso.mse_path_, ':')

plt.plot(alphas, model_lasso.mse_path_.mean(axis=1), 'k',     #Moyenne
         label='Moyenne', linewidth=2)

plt.axvline(model_lasso.alpha_, linestyle='--', color='k',    #alpha: estimation CV
            label='alpha: estimation CV')

plt.legend()

plt.xlabel('Alpha')
plt.ylabel('Mean square error')
plt.title('Mean square error pour chaque échantillon ')
plt.show()

# - 4 ---------------------------------------  Predict + score + MSE 

pred_test = model_lasso.predict(X_test)                 #predict

print("score test:", model_lasso.score(X_test, y_test))   #score
print("mse test:", mean_squared_error(pred_test, y_test)) #mean_squared_error

----

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------
##  Modèles de régression avec scikit-learn : Qui sera le MVP ? - Elastic Net
# ------------------------------------------------------------------------------------------------------------------------------

# - A ------------------------------------------------------------------------------ Dataset

# - 1 ---------------------------------------  Import
import numpy as np
import pandas as pd

from sklearn import model_selection, preprocessing

from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

# - 2 ---------------------------------------  traitement df
df = pd.read_csv (  'nba_2013.csv'  )   #import 

nba.describe()    #connaitre son df
nba.info()
nba.shape

nba.index = nba [ ['player', 'bref_team_id'] ]  #modif du df (index)

#Supprimez les lignes contenant des valeurs manquantes.
nba.isnull().sum()  #check val na
nba.dropna( how='any',inplace = True )  #supprime val na

#Affiche les différentes modalités prises par la variables pos.
nba['pos'].mode()[0]   #modalite la plus courante
nba['pos'].value_counts()   #modalites de la col pos

#Supprime les lignes contenant des valeurs aberrantes pour la variable pos
nba = nba[nba ['pos'] != 'G']

pl_pos = nba ['pos'] # var en string

nba = nba.drop ( ['season','player', 'bref_team_id', 'pos' ] , axis = 1 )   #Supprime les colonnes 

scaler = preprocessing.StandardScaler().fit (nba)        #fct de normalisation les colonnes de nba
nba[nba.columns] = pd.DataFrame(scaler.transform(nba), index= nba.index)  #utilisation de la fct de normalisatoin

nba = nba.join( pd.get_dummies (pl_pos, prefix = 'Pos') )  #var en string -> get dummies


# - 3 ---------------------------------------  Separation target/data
#separation du df en cible / feature 
target = nba ['pts']                                   #target  ou y
data   = nba.drop(  ['pts','Pos_SG'] , axis = 1  )    #data ou x



# - B ------------------------------------------------------------------------------ Modele LinearRegression


# - 1 ---------------------------------------  train_test_split
#échantillons d'apprentissage et de test 
x_train,x_test,y_train,y_test = train_test_split ( data , target , test_size = 0.2, random_state=101 )


# - 2 ---------------------------------------  correlation 
import seaborn as sns

plt.figure ( figsize=(16,15) )
sns.heatmap (  nba.corr() , annot=True, cmap="RdBu_r", center =0) #heatmap + corr()


# - 3 ---------------------------------------  modèle de régression linéaire simple 
from sklearn.linear_model import LinearRegression   #import fct
model_1 = LinearRegression ()     #creation de modele

model_1.fit ( X_train_mp,y_train )   #fit   #ca marchera pas il faut 

# - 4 ---------------------------------------  m n matrice target / Data
#echantill train 
X_train_mp = X_train['mp']   
X_train_mp.shape   #(368,)
X_train_mp = np.expand_dims( X_train_mp , axis = 1 ) #(368,1)
X_train_mp  = np.hstack( ( X_train_mp, np.ones(X_train_mp.shape)) ) #Fusion  #(368,2)
y_train.shape   #(368,)
#test
X_test_mp = X_test['mp'] 
X_test_mp.shape  #(92,)
X_test_mp = np.expand_dims( X_test_mp, axis = 1 )  #new dimention 
X_test_mp.shape  #(92, 1)
X_test_mp = np.hstack( (X_test_mp , np.ones( X_test_mp.shape)) )  #fusion 
y_test.shape    #(92,)

# - 4 ---------------------------------------  score R² et le RMSE
print (model_1.score ( X_train_mp , y_train  ) )   #score train
print (model_1.score ( X_test_mp , y_test  ) )   #score test

pred = model_1.predict (X_train_mp)      #predcition train
pred_test = model_1.predict(X_test_mp)  #predcition test

print("rmse train:", np.sqrt(mean_squared_error(y_train, pred)))      # rmse train
print("rmse test: ", np.sqrt(mean_squared_error(y_test, pred_test)))  # rmse test

# - B ------------------------------------------------------------------------------ ElasticNetCV

# - 1 ---------------------------------------  modèle + fit 
from sklearn.linear_model import  ElasticNetCV  #imprt fct 

# modèle de régression Elastic Net
model_en =  ElasticNetCV ( l1_ratio = (0.1, 0.25, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 0.99) , 
                          alphas= (0.001,0.01,0.02,0.025,0.05,0.1,0.25,0.5,0.8,1.0) )

model_en.fit (X_train , y_train )  #train

# - 2 ---------------------------------------  intercept et coefficients estimé
#l'intercept 
model_en.intercept_

#coefficients estimés pour chaque variable de data.
pd.DataFrame (  {'coef' :model_en.coef_}, index = data.columns)   


# - 2 ---------------------------------------  courbe de la moyenne des erreurs MSE par validation croisée 
#en fonction des valeurs de  α
alphas = model_en.alphas_

plt.figure(figsize = (10,10))

for i in range(model_en.mse_path_.shape[0]) :
    plt.plot(alphas, model_en.mse_path_[i,:,:].mean(axis=1),
         label='Moyenne pour l1_ratio= %.2f' %model_en.l1_ratio[i], linewidth=2)

plt.legend()

plt.xlabel('Alpha')
plt.ylabel('Mean squared error')
plt.title('Mean squared error pour chaque $\lambda$')
plt.show()

# - 3 ---------------------------------------  mse + predict + score
#prédictions du modèle
pred_train =  model_en.predict ( X_train )
pred_test  =  model_en.predict ( X_test)

#erreurs quadratiques moyennes
print(np.sqrt(mean_squared_error(y_train, pred_train)))
print(np.sqrt(mean_squared_error(y_test, pred_test)))

print (model_en.score ( X_train  , y_train))
print (model_en.score ( X_test , y_test))


# - 4 --------------------------------------- moyenne et l'écart type calculés par l'objet scaler 
#et utilisés pour transformer les données
# la moyenne et l'écart type calculés par l'objet scaler
moy = scaler.mean_[-1]   
ec = scaler.scale_[-1]
print("moyenne :", moy)
print("ecart-type", ec)

# - 5 ---------------------------------------  points observés pour chaque joueur, et les points prédits par modèle Elastic Net
pd.DataFrame({'points_obsérvés': (y_test*ec)+moy, 'points_predits' : np.round((pred_test*ec)+moy)}, index = X_test.index).head(7)

L'objectif de ce test consiste à prédire, à l'aide de modèles de régressions, le prix d'une habitation en fonction de variables pertinentes représentant différentes caractéristiques architecturales, géographiques et propres au voisinage.

Le jeu de données utilisé contient de nombreux attributs permettant de décrire 1460 habitations en fonction de plusieurs caractéristiques ainsi qu'une variable cible: 'SalePrice' contenant le prix de vente du bien en question.

Le jeu de données est à lire dans le fichier "house_price.csv" (les colonnes sont séparées par des ';')

Exécuter la cellule suivante pour importer les packages nécessaire pour la suite de l'exercice

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, LassoCV

import matplotlib.pyplot as plt
%matplotlib inline

Lire le fichier "house_price.csv" dans un data frame appelé hp

In [ ]:
## Insérez votre code ici


df = pd.read_csv ('house_price.csv',sep = ',')

col_num = df.select_dtypes ( include = ['float64', 'int64'] ).columns
col_qual = df.select_dtypes ( include = ['object'] ).columns

Remplacer, pour chaque variable, les valeurs manquantes par la moyenne de la variable

In [ ]:
for i in df[col_num] :
     df[i].fillna(df[i].mean() ,inplace = True)


for i in df[col_qual] :
     df[i].fillna(df[i].mode() [0],inplace = True)

Centrer et réduire les variables numériques du data frame, à l'aide d'une instance de la classe preprocessing.StandardScaler

In [ ]:


df [liste_num]

#scaler = preprocessing.StandardScaler().fit (df)      
#df[df.columns] = pd.DataFrame(scaler.transform(df), index= df.index) 

Transformer chaque variable catégorielle en variables indicatrices.

In [ ]:
df = df.join( pd.get_dummies (df[col_qual], prefix = 'gd') ) 

df = df.drop(df[col_qual], axis = 1)


df.shape  # (1460, 290)

Séparer la variable cible 'SalePrice' dans target et le reste des variables dans feats

In [ ]:
target = df['SalePrice']
feats  = df.drop (['SalePrice'], axis = 1)

Séparer les données en un ensemble d'apprentissage (X_train, y_train) et un ensemble de test(X_test, y_test), avec 20% des données originales pour le test

In [ ]:
X_train,X_test,y_train,y_test = train_test_split ( feats , target , test_size = 0.2 )


Créer un une fonction rmse_cv, qui calcule pour un modèle donné, la racine de l'erreur quadratique moyenne obtenue par validation croisée à 5 échantillons, grâce à la fonction cross_val_score de sklearn.model_selection.

In [ ]:
from  sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def rmse_cv(mod,X,y):  
    return np.sqrt( cross_val_score(mod,X,y, cv=5, scoring=make_scorer(mean_squared_error)).mean() )


Créer un vecteur alphas contenant les valeurs [0.01, 0.05, 0.1, 0.3, 0.8, 1, 5, 10, 15, 30, 50].
Afficher dans un graphique, le RMSE retourné par la fonction rmse_cv appliquée à un modèle de régression Ridge(alpha = αα ) pour chaque  αα  appartenant à alphas.

In [ ]:
## Insérez votre code ici

from sklearn.linear_model import RidgeCV  #import modele RidgeCV 

alpha = [0.01, 0.05, 0.1, 0.3, 0.8, 1, 5, 10, 15, 30, 50 ]

x= []
y = []

for i in range (len ( alpha)) : 
    ridge = Ridge(alpha[i])
    x.append (alpha [i])
    y.append (rmse_cv ( ridge, X_train,y_train  ))

    
plt.plot (x,y)
plt.xlabel ('alpha')
plt.ylabel ('rmse')

À partir du graphique affiché, créer un modèle de régression Ridge performant
Tester les performances de ce modèle sur X_test

In [ ]:
## Insérez votre code ici
# le alpha avec le rmse le plus petit 10
ridge = Ridge(alpha = 10)
ridge.fit ( X_train,y_train )
ridge.score(X_test,y_test)

Créer un modèle de régression Lasso qui choisira le paramètre  αα  parmi [10, 1, 0.1, 0.001, 0.0005] par validation croisée, grâce à la fonction LassoCV
Afficher les performances du modèle sur l'échantillon de test

In [ ]:
model_lasso = LassoCV(cv=10, alphas = [10, 1, 0.1, 0.001, 0.0005] )
model_lasso.fit(X_train, y_train)
model_lasso.score(X_test, y_test)


À partir des coefficients retournés par le modèle de régression Lasso, afficher le nombre de variables gardés et le nombre de variables éliminés par le modèle 

In [ ]:
lasso_coef = model_lasso.coef_

plt.plot(range(len(feats.columns)), lasso_coef)
plt.xticks(range(len(feats.columns)), feats.columns.values, rotation=70)
plt.ysize ( )
plt.show()


Afficher les variables les plus importantes du modèle, relativement aux coefficients calculés

In [ ]:
a = pd.DataFrame ({'coef': model_lasso.coef_}, index = feats.columns)

